Run the following in the command line, before proceeding to the examples
> pip install quick_pp

> quick_pp api-server

View the API documentation at localhost:8888/docs or localhost:8888/redoc

In [ ]:
%load_ext autoreload
%autoreload 2

# Setup working directory to the project root.
import os
cwd = os.getcwd()
project_name = 'quick_pp'
os.chdir(fr'{cwd.split(project_name)[0] + project_name}')
print(os.getcwd())

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import requests

import quick_pp.las_handler as las
from quick_pp.qaqc import badhole_flagging, handle_outer_limit, neu_den_xplot_hc_correction
from quick_pp.plotter.plotter import plotly_log

In [ ]:
def make_api_request(df_dict: dict,
                     endpoint: str,
                     session: requests.Session,
                     files: list = [],
                     verify: bool = False) -> list:
    """""
    This method sends a prediction request to the FastAPI Swagger UI. It uses the provided DataFrame
    dictionary, field, use case, method, and model to make the request. It also uses the provided session
    and verify flag.

    Args:
        df_dict (dict): The DataFrame dictionary to use for the prediction request.
        field (str): The field to use for the prediction request.
        use_case (str): The use case to use for the prediction request.
        method (str): The method to use for the prediction request.
        model (str): The model to use for the prediction request.
        session (requests.Session, optional): The session to use for the prediction request.
        If not provided, a new session will be created. Defaults to None.
        verify (bool, optional): The verify flag to use for the prediction request.
        If set to False, the SSL certificate will not be verified. Defaults to False.

    Returns:
        list: The prediction result if the request is successful.

    Raises:
        AssertionError: If the first key in the DataFrame dictionary is not "data".
    """
    assert "data" in df_dict.keys()

    # Define model API server to the FastAPI Swagger UI.
    model_server = {
        "local": "http://localhost:8888",
    }

    headers = {
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    try:
        # Create url to access the model API.
        url = f"{model_server['local']}/quick_pp/{endpoint}"
        print(f"Requesting API to {url}")

        # Get the response from the API.
        response = session.post(url=url,
                                json=df_dict,
                                # headers=headers,
                                files=files,
                                verify=verify)
        status = response.status_code

        if status == 200:
            return response.json()
        else:
            print(f"[make_api_request] Error | {response.text} ")

    except Exception as e:
        print(f"[make_api_request] Error | {e} ")


session = requests.Session()

In [ ]:
# Make the prediction request via API.
files = [
    ('files', open(r'data\01_raw\36_7-3.las', 'rb'))
]
api_data_response = make_api_request(df_dict={'data': []}, endpoint='las_handler', files=files, session=session)
print(f"API response: {api_data_response}")

In [ ]:
df = pd.read_parquet(fr"../{api_data_response['file_paths'][0]}")

df['RT'] = df['RDEP']  # Use RDEP instead of RT

# Mask outside threshold
df = handle_outer_limit(df, True)

# Flag bad hole
df = badhole_flagging(df) if 'CALI' in df.columns else df
# Prepare the data for the prediction request.
df.interpolate(inplace=True)  # Interpolate null values
df.dropna(inplace=True)  # Drop remaining null values
clear_output()

In [ ]:
df.columns

# Lithology

In [ ]:
data_dict = {
    "dry_sand_point": [
        -0.02,
        2.65
    ],
    "dry_silt_point": [
        0,
        2.68
    ],
    "dry_clay_point": [
        0,
        2.7
    ],
    "fluid_point": [
        1,
        1
    ],
    "wet_clay_point": [
        0,
        0
    ],
    "method": "ssc",
    "silt_line_angle": 117,
}

In [ ]:
df_copy = pd.DataFrame()
df_copy[['nphi', 'rhob']] = df[['NPHI', 'RHOB']]
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary
df_dict.update(data_dict)

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='lithology/ssc', session=session)
df_ssc = df.copy()
df_ssc[['VSAND', 'VSILT', 'VCLAY']] = pd.DataFrame(api_data_response)[['VSAND', 'VSILT', 'VCLAY']]

In [ ]:
fig = plotly_log(df_ssc)
fig.show()
# fig.write_html('plot.html')

## Hydrocarbon Correction

In [ ]:
df_copy = pd.DataFrame()
df_copy[['nphi', 'rhob', 'gr']] = df[['NPHI', 'RHOB', 'GR']]
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary
df_dict.update(data_dict)
df_dict.update({'dry_clay_point': (0.33, 2.7), 'corr_angle': 50})

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='lithology/hc_corr', session=session)
df_corr = df.copy()
df_corr[['VSAND', 'VSILT', 'VCLAY']] = pd.DataFrame(api_data_response)[['VSAND', 'VSILT', 'VCLAY']]

# Porosity

In [ ]:
# df_dict = pd.DataFrame()
# df_dict[['nphi', 'rhob', 'gr']] = df[['NPHI', 'RHOB', 'GR']]
# df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary
# df_dict.update(data_dict)
# df_dict.update({'dry_clay_point': (0.33, 2.7), 'corr_angle': 50})

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='porosity/den', session=session)
df_corr['PHID'] = pd.DataFrame(api_data_response)['PHID']

In [ ]:
# Make the prediction request via API.
df_dict.update({'dry_silt_point': (0.15, 2.68)})
api_data_response = make_api_request(df_dict=df_dict, endpoint='porosity/neu_den', session=session)
df_corr['PHIT'] = pd.DataFrame(api_data_response)['PHIT']

In [ ]:
df_corr.plot(x='DEPTH', y=['PHIT', 'PHID'], figsize=(20, 5))

# Water Saturation

In [ ]:
# Estimate the temperature gradient

df_copy = pd.DataFrame()
df_copy[['tvdss']] = df[['DEPTH']]
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/temp_grad', session=session)
df_corr['TEMP_GRAD'] = pd.DataFrame(api_data_response)['TEMP_GRAD']

In [ ]:
# Estimate the formation water resistivity

df_copy = pd.DataFrame()
df_copy[['temp_grad']] = df_corr[['TEMP_GRAD']].dropna()
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary
df_dict.update({'water_salinity': 30000})

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/rw', session=session)
df_corr['RW'] = pd.DataFrame(api_data_response)['RW']

In [ ]:
# Estimate archie water saturation

df_copy = pd.DataFrame()
df_copy[['rt', 'rw', 'phit']] = df_corr[['RT', 'RW', 'PHIT']].dropna()
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/archie', session=session)
df_corr['SWT_A'] = pd.DataFrame(api_data_response)['SWT']

In [ ]:
# Estimate the conductivity factor, b

df_dict = pd.DataFrame()
df_dict[['temp_grad', 'rw']] = df_corr[['TEMP_GRAD', 'RW']].dropna()
df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/b_waxman_smits', session=session)
df_corr['B'] = pd.DataFrame(api_data_response)['B']

In [ ]:
# Estimate the cation exchange capacity, qv

df_dict = pd.DataFrame()
df_dict[['vcld', 'phit']] = df_corr[['VCLAY', 'PHIT']].dropna()
df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary
df_dict.update({'rho_clay': 2.65, 'cec_clay': 0.062})

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/estimate_qv', session=session)
df_corr['QV'] = pd.DataFrame(api_data_response)['QV']

In [ ]:
# Estimate waxman-smits water saturation
df_corr['M'] = 2
df_dict = pd.DataFrame()
df_dict[['rt', 'rw', 'phit', 'b', 'qv', 'm']] = df_corr[['RT', 'RW', 'PHIT', 'B', 'QV', 'M']].dropna()
df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='saturation/waxman_smits', session=session)
df_corr['SWT'] = pd.DataFrame(api_data_response)['SWT']

In [ ]:
df_corr.plot(x='DEPTH', y=['SWT', 'SWT_A'], figsize=(20, 5), ylim=(0, 1.1))

# Permeability

In [ ]:
# Estimate Choo's permeability

df_dict = pd.DataFrame()
df_dict[['vcld', 'vsilt', 'phit']] = df_corr[['VCLAY', 'VSILT', 'PHIT']].dropna()
df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary
# df_dict.update(data_dict)

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='permeability/choo', session=session)
df_corr['PERM'] = pd.DataFrame(api_data_response)['PERM']

In [ ]:
# Estimate other's permeability
constant = df_corr['VCLAY']**1.75
df_corr['SWIRR'] = constant / df_corr['PHIT']
df_dict = pd.DataFrame()
df_dict[['phit', 'swirr']] = df_corr[['PHIT', 'SWIRR']].dropna()
df_dict = {"data": df_dict.to_dict(orient="records")}  # Convert data_df to dictionary
# df_dict.update(data_dict)

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='permeability/timur', session=session)
df_corr['PERM_T'] = pd.DataFrame(api_data_response)['PERM']

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='permeability/tixier', session=session)
df_corr['PERM_TX'] = pd.DataFrame(api_data_response)['PERM']

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='permeability/coates', session=session)
df_corr['PERM_C'] = pd.DataFrame(api_data_response)['PERM']

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='permeability/kozeny_carman', session=session)
df_corr['PERM_KC'] = pd.DataFrame(api_data_response)['PERM']

In [ ]:
df_corr.plot(x='DEPTH', y=['PERM', 'PERM_KC', 'PERM_T', 'PERM_C', 'PERM_TX'], figsize=(20, 5), logy=True)

# Rock Type

In [ ]:
std = df_corr['VCLAY'].describe()['std']
standard_q = [0.2, 0.4, 0.6]
proportion = [pct - std for pct in standard_q]
proportion = standard_q if any([p < 0.15 for p in proportion]) else proportion
q_dict = df_corr['VCLAY'].quantile(proportion).to_dict()
q_dict

In [ ]:
df_corr['ROCK_FLAG'] = np.where(df_corr['VCLAY'] < list(q_dict.values())[0], 1,
                               np.where(df_corr['VCLAY'] < list(q_dict.values())[1], 2,
                                        np.where(df_corr['VCLAY'] < list(q_dict.values())[2], 3, 4)))

# Reservoir Summary

In [ ]:
df_copy.head()

In [ ]:
# Calculate reservoir summary
df_corr['ZONES'] = 'ALL'
df_copy = pd.DataFrame()
df_copy[['depth', 'vcld', 'phit', 'swt', 'perm', 'zones']] = df_corr[
    ['DEPTH', 'VCLAY', 'PHIT', 'SWT', 'PERM', 'ZONES']].dropna()
df_dict = {"data": df_copy.to_dict(orient="records")}  # Convert data_df to dictionary
# df_dict.update(data_dict)
df_dict.update({'cut_offs': dict(VSHALE=0.4, PHIT=.05, SWT=.8)})

# Make the prediction request via API.
api_data_response = make_api_request(df_dict=df_dict, endpoint='ressum', session=session)
ressum_df = pd.DataFrame(api_data_response)

In [ ]:
ressum_df

In [ ]:
fig = plotly_log(df_corr)
fig.show(config={'scrollZoom': True, 'displayModeBar': True})
# fig.write_html('plot.html')